In [1]:
import numpy as np
import pandas as pd
import sys


In [38]:
dfImport = pd.read_csv("data/national_names.csv")


In [39]:
df = dfImport
df = df.sort_values(['Name'])


In [40]:
df = df.loc[df["Year"] == 2012].reset_index()
df = df.iloc[0:300]


In [41]:
ls_Components = df["Name"].tolist()
len(ls_Components)

300

In [55]:
DISTANCE_LENGTH = 2

def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    return (matrix[size_x - 1, size_y - 1])

def form_groups(data):

    grs = list() 
    for name in data:
        for g in grs:
            if all(levenshtein(name, w) < DISTANCE_LENGTH for w in g):
                g.append(name)
                break
        else:
            grs.append([name, ])

    print('output groups:', len(grs))
    return list(grs)

def allocate_groups(grs, data):
    return_list = list()
    for checkagainst in data:
        for i in range(0, len(grs)):
                if checkagainst in grs[i]:
                    return_list.append(i)
    return return_list

In [56]:
grs = form_groups(ls_Components)
allocated_groups = allocate_groups(grs, ls_Components)

df["allocated_group"] = allocated_groups

output groups: 176


In [57]:
df.loc[df["allocated_group"] == 10]

,index,Id,Name,Year,Gender,Count,allocated_group
14,1752002,1752003,Aadish,2012,M,11,10
17,1755705,1755706,Aadith,2012,M,6,10


In [60]:
#allocated_groups

In [59]:
df

,index,Id,Name,Year,Gender,Count,allocated_group
0,1751999,1752000,Aaban,2012,M,11,0
1,1742312,1742313,Aabha,2012,F,5,1
2,1752000,1752001,Aadam,2012,M,11,2
3,1752001,1752002,Aadan,2012,M,11,0
4,1751575,1751576,Aadarsh,2012,M,12,3
...,...,...,...,...,...,...,...
295,1735897,1735898,Aberdeen,2012,F,10,172
296,1737679,1737680,Abernathy,2012,F,8,173
297,1740437,1740438,Abha,2012,F,6,174
298,1747868,1747869,Abhay,2012,M,40,174
